In [374]:
import os
import pandas as pd
import numpy as np
import json
import os.path
import argparse
from re import search

### !!! Using temporary data directory
#run_dir = "/Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT"

#single_file = "" # "0306, --- WILSON of Bourtrees"

#is_debugging = True
#error_msgs = []


In [375]:
template = """<!DOCTYPE html>
<html lang="en">
    <meta charset="utf-8" />

    <style type="text/css">
        #graph{ width: 100%;; }
        body {
            font: 10px sans-serif;
        }
        .linage {
            fill: none;
            stroke: #000;
        }
        .marriage {
            fill: none;
            stroke: black;
        }
        .marriageNode {
            background-color: black;
            border-radius: 50%;
        }
        .man {
            background-color: #fff;
            border: 1px solid #aaa;
            box-sizing: border-box;
        }
        .in_WIN {
            background-color: red;
            border: 1px solid #aaa;
            box-sizing: border-box;
        }
        .in_LWH {
            background-color: green;
            border: 1px solid #aaa;
            box-sizing: border-box;
        }
        .woman {
            background-color: pink;
            border-style: solid;
            border-width: 1px;
            box-sizing: border-box;
        }
        .unknown {
            background-color: #ddd;
            border-style: solid;
            border-width: 1px;
            box-sizing: border-box;
        }
        .emphasis{
            font-style: italic;
        }
        p {
            padding:0;
            margin:0;
        }
        svg {
            border-style: solid;
            border-width: 1px;
        }
    </style>

    <script src="https://d3js.org/d3.v4.js"></script>
    <script src="https://cdn.jsdelivr.net/lodash/4.17.4/lodash.min.js"></script>
    <script src="https://cdn.jsdelivr.net/npm/d3-dtree@2.4.1/dist/dTree.min.js"></script>

    <body>
        <h1>{filename}</h1>
        <div id="graph"></div>

        <script>
            treeJson = d3.json("{json_filepath}", function(error, treeData) {
                dTree.init(treeData, {
                    target: "#graph",
                    debug: true,
                    hideMarriageNodes: true,
                    marriageNodeSize: 5,
                    height: 800,
                    width: 1200,
                    callbacks: {
                        nodeClick: function(name, extra) {
                            alert('Click: ' + name);
                        },
                        nodeRightClick: function(name, extra) {
                            alert('Right-click: ' + name);
                        },
                        textRenderer: function(name, extra, textClass) {
                            if (extra && extra.nickname)
                                name = name + " (" + extra.nickname + ")";
                            return "<p align='center' class='" + textClass + "'>" + name + "</p>";
                        },
                        marriageClick: function(extra, id) {
                            alert('Clicked marriage node' + id);
                        },
                        marriageRightClick: function(extra, id) {
                            alert('Right-clicked marriage node' + id);
                        },
                    }
                });
            });
        </script>
    </body>
</html>"""

In [376]:
def build_tree2( parent, structure, annotations, annotation_column ):

    if is_debugging: print ( "+ Tree building for %s %s" % ( parent[0], parent[1] ) )
        
    ### --- Identifying the annotation for the current 'parent' node
    parent_annotations = ( annotations
                          .drop_duplicates(subset=['pers_id', 'pers_id_context', annotation_column ] )
                          .query( 'pers_id=="%s" & pers_id_context=="%s"' % ( parent[0], parent[1] ) )
                          .query( '%s=="Yes"' % annotation_column)
                         )
    
    this_class = "man"
    if parent_annotations.shape[0] > 0:
        this_class = annotation_column
    
    if is_debugging: print ( "> Node identified as class '%s'" % this_class )
    
    node = {}
    node["name"] = parent[0]
    node["class"] = this_class
    
    if "(" in parent[1]:
        node["extra"] = {}
        node["extra"]["nickname"] = parent[1]
    
    structure_children = structure.query( 'pers2_id=="%s" & pers2_id_context=="%s"' % ( parent[0], parent[1] ) )
    children_ids = structure_children[["pers1_id","pers1_id_context"]].to_records(index=False)
    # TODO: make sure this is just unique children...
    
    if is_debugging: print ( "> Node has %d children" % len( children_ids ) )

    if len(children_ids)>0:
        m = {}
        m["spouse"] = {
            "name": "Unknown",
            "class": "unknown",
        }
        m["children"] = []
        for child in children_ids:
            m["children"].append( build_tree2( child, structure, annotations, annotation_column ) )
        node["marriages"] = [ m ]

    return node

In [377]:
def convert_DAT_to_tree2( peorel_file, peopla_file ):

    ##########################################################
    ### STEP 1: read in and process data from PEO/REL file ###
    ##########################################################

    if is_debugging: print( "+ Reading PEO/REL file\n> " + peorel_file )
    PEOREL_in = pd.read_csv(peorel_file, "\t", header=0)
    if is_debugging: print ( "> Rows = %d, Columns = %d\n" % PEOREL_in.shape )

    ### Filter dataframe for son/daughter/child relationships
    ### https://stackoverflow.com/questions/12065885/filter-dataframe-rows-if-value-in-column-is-in-a-set-list-of-values/46460307#46460307
    rel_list = ["SON","DAUG","CHILD"]
    PEOREL_d = ( PEOREL_in
                .query( "rel in @rel_list" )
                .drop_duplicates(subset=['src_ref', 'src_linenum']))
    PEOREL_d[ "pers1_id_context" ] = PEOREL_d[ "pers1_id_context" ].astype(str)
    PEOREL_d[ "pers2_id_context" ] = PEOREL_d[ "pers2_id_context" ].astype(str)

    if is_debugging: print( "+ Filtering PEO/REL data for " + "/".join( rel_list ) )
    if is_debugging: print( "+ Removing duplicationes [of src_ref/src_linenum]" )
    if is_debugging: print ( "> Rows = %d, Columns = %d\n" % PEOREL_d.shape )

    ##########################################################
    ### STEP 2: read in and process data from PEO/PLA file ###
    ###         (if present)                               ###
    ##########################################################
    locations_of_interest = ["LWH","WIN"]
    PEOPLA_annotated = ()
    
    if os.path.exists(peopla_file):
        if is_debugging: print( "+ Reading PEO/PLA file\n> " + peopla_file )
        PEOPLA_in = pd.read_csv( peopla_file, "\t", header=0 )
        PEOPLA_in[ "pers_id_context" ] = PEOPLA_in[ "pers_id_context" ].astype(str)
        PEOPLA_in[ "place_id" ] = PEOPLA_in[ "place_id" ].astype(str)
        if is_debugging: print ( "> Rows = %d, Columns = %d\n" % PEOPLA_in.shape )

        PEOPLA_annotated = PEOPLA_in
        
        ### --- Adding flags for locations of interest 
        for location in locations_of_interest:
            new_col = "in_" + location
            if is_debugging: print ( "+ Adding annotation '%s' for place_ids containing '%s'" % (new_col, location))
            PEOPLA_annotated[new_col] = ( PEOPLA_annotated['place_id']
                                         .apply( lambda x : detect_country(x,location) ) )
    else:
        ### --- If no PEOPLA file exists, generate an empty one,
        ### --- adding default values of "No" for the location annotations
        ### --- that we're interested in 
        group1_people = ( PEOREL_d[["src_ref",
                                       "src_pp",
                                       "src_pp_original",
                                       "src_linenum",
                                       "src_linenum_original",
                                       "pers1_id",
                                       "pers1_id_context"]]
                         .rename(columns={'pers1_id': 'pers_id', 'pers1_id_context': 'pers_id_context'}) )
        group2_people = ( PEOREL_d[["src_ref",
                                       "src_pp",
                                       "src_pp_original",
                                       "src_linenum",
                                       "src_linenum_original",
                                       "pers2_id",
                                       "pers2_id_context"]]
                         .rename(columns={'pers2_id': 'pers_id', 'pers2_id_context': 'pers_id_context'}) )
        
        PEOPLA_annotated = ( group1_people
                            .append( group2_people )
                            .drop_duplicates(subset=['src_ref','src_linenum','pers_id', 'pers_id_context'] ) )
        
        for location in locations_of_interest:
            new_col = "in_" + location
            if is_debugging: print ( "+ No PEOPLA data, using default values in '%s'" % (new_col))
            PEOPLA_annotated[new_col] = "No"

    ##########################################################
    ### STEP 4: Check for multiple root nodes ################
    ##########################################################
    ### Identifying how many root nodes there are, so that an error
    ### message can be generated and the file name recorded.
    ###
    ### This step is done by merging the database with itself - any 
    ### person who does not have a "parent" in the tree structure
    ### will have NA in pers1_id_r column.  By filtering the merged
    ### dataset for NAs in this column, we will find the root nodes.
    ### Note that I've used a hacky way of identifying NAs here
    ### (NaN does not equal itself).
    ##########################################################

    root_nodes_d = ( PEOREL_d
                    .merge( PEOREL_d[["pers1_id","pers1_id_context"]],
                           how="left",
                           left_on=["pers2_id","pers2_id_context"],
                           right_on=["pers1_id","pers1_id_context"],
                           suffixes=('', '_r') )
                    .query( "pers1_id_r != pers1_id_r" ) # Hacky way to find NaNs
                    .drop_duplicates(subset=['pers2_id', 'pers2_id_context'] ) )

    ### to_records() converts a DataFrame to a NumPy record array.
    root_nodes = root_nodes_d[["pers2_id", "pers2_id_context", "src_linenum"]].to_records(index=False)

    if is_debugging: print ( "> There are %d root nodes" % len( root_nodes ) )
        
    if len( root_nodes ) > 1:
        if is_debugging: print ( "<!> WARNING: Multiple trees (n=%d)" % len( root_nodes ) )
        for pid in root_nodes:
            print( "  - %s: %s %s" % ( pid[2], pid[0], pid[1] ) )
        error_msgs.append( peorel_file )

    i = 1
    
    for this_root in root_nodes:
        if len( root_nodes ) > 1: identifier = "_%s" % i
        else: identifier = ""

        tree = build_tree2( this_root, PEOREL_d, PEOPLA_annotated, "in_WIN" )

        stub = peorel_file.split("03_^DAT",1)[1]
        dirpath = peorel_file.split("03_^DAT",1)[0]
        vis_dir = "%s/%s" % ( dirpath, "04_^VIS" )
        trees_dir = "%s/%s" % ( vis_dir, "^trees" )

        if is_debugging:
            print( "+ Writing to visualisation dir: %s" % vis_dir )
            print( "+ Writing to trees dir: %s" % trees_dir )

        if not os.path.exists( vis_dir ): os.mkdir( vis_dir )
        if not os.path.exists( trees_dir ): os.mkdir( trees_dir )

        json_filepath = "%s/%s" % ( trees_dir, stub.replace(".PEO_REL.tsv", "%s.json" % identifier) )
        html_filepath = "%s/%s" % ( trees_dir, stub.replace(".PEO_REL.tsv", "%s.html" % identifier) )
        with open( json_filepath, "w" ) as w:
            w.write( json.dumps( [ tree ] ) )
        with open( html_filepath, "w" ) as w:
            w.write( template.replace( "{json_filepath}", json_filepath ).replace( "{filename}", stub ).replace( ".PEO_REL.tsv", "" ) )
        i += 1


In [378]:
def detect_country(x, country):
    if x == "nan":
        return( "No")
    elif x.find(country) != -1:
        return( "Yes" )
    else:
        return( "No" )

In [387]:
### !!! Using temporary data directory
#run_dir = "/Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT"
#single_file = "" # "0306, --- WILSON of Bourtrees"


if __name__ == "__main__":
    
    parser = argparse.ArgumentParser(description="Generating tree visualisations from PEO/REL and PEO/PLA data sources.")

    parser.add_argument("run_dir",
                        nargs='?',
                        help="The directory in which to search for PEO/REL and PEO/PLA data sources",
                        default=".")
    parser.add_argument("-v",
                        help="Print additional messages at run time",
                        action='store_true')

    args = parser.parse_args()

    is_debugging = False
    if args.v: is_debugging = True
    
    error_msgs = []

    for root, dirs, files in os.walk( run_dir ):
        for fname in files:
            if fname.endswith( ".PEO_REL.tsv" ) and "03_^DAT" in root:
                #if not single_file in fname: continue
                peorel_file = "%s/%s" % ( root, fname )
                peopla_file = peorel_file.replace("PEO_REL", "PEOPLA", 1)
                convert_DAT_to_tree2( peorel_file, peopla_file )
                print( "--------------------------------------------------------" )

    print( "%s files with more than one tree root." % len(error_msgs) )

+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0095, The factory.jpg.txt.PEO_REL.tsv
> Rows = 0, Columns = 10

+ Filtering PEO/REL data for SON/DAUG/CHILD
+ Removing duplicationes [of src_ref/src_linenum]
> Rows = 0, Columns = 10

+ Reading PEO/PLA file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0095, The factory.jpg.txt.PEOPLA.tsv
> Rows = 5, Columns = 44

+ Adding annotation 'in_LWH' for place_ids containing 'LWH'
+ Adding annotation 'in_WIN' for place_ids containing 'WIN'
> There are 0 root nodes
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0171, --- FLEMING of Newmill [+].jpg.txt.PEO_REL.tsv
> Rows = 26, Columns = 10

+ Filtering PEO/REL data for SON/DAUG/CHILD
+ Removing duplicationes [of src_ref/src_linenum]
> Rows = 13, Columns = 10

+ Reading PEO/PLA file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0171, --- FLEMING of N

> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Jean 41
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Mary 49
> Node identified as class 'man'
> Node has 0 children
+ Writing to visualisation dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS
+ Writing to trees dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS/^trees
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0085_2, --- DUNLOP of Boreland.jpg.txt.PEO_REL.tsv
> Rows = 0, Columns = 10

+ Filtering PEO/REL data for SON/DAUG/CHILD
+ Removing duplicationes [of src_ref/src_linenum]
> Rows = 0, Columns = 10

+ No PEOPLA data, using default values in 'in_LWH'
+ No PEOPLA data, using default values in 'in_WIN'
> There are 0 root nodes
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat

> Node has 0 children
+ Tree building for SPEIR, Hugh 69
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SPEIR, Elizabeth 77
> Node identified as class 'man'
> Node has 2 children
+ Tree building for BARR, Robert 83
> Node identified as class 'man'
> Node has 0 children
+ Tree building for BARR, Elizabeth 93
> Node identified as class 'man'
> Node has 0 children
+ Writing to visualisation dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS
+ Writing to trees dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS/^trees
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0091, --- PRATT of Kirkaldie [+].jpg.txt.PEO_REL.tsv
> Rows = 4, Columns = 10

+ Filtering PEO/REL data for SON/DAUG/CHILD
+ Removing duplicationes [of src_ref/src_linenum]
> Rows = 4, Columns = 10

+ Reading PEO/PLA file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0091, --- P

> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Marion 89
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Mary 93
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Hugh 100
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Robert 123
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Jean 134
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Margaret 160
> Node identified as class 'man'
> Node has 0 children
+ Writing to visualisation dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS
+ Writing to trees dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS/^trees
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0096_2, --- COCHRANE of Ladyland [+].jpg.txt.PEO_REL.tsv
> Rows = 52, Columns = 10

+ Fil

> Node identified as class 'man'
> Node has 0 children
+ Tree building for AITKEN, James 71
> Node identified as class 'man'
> Node has 2 children
+ Tree building for AITKEN, Janet 78
> Node identified as class 'man'
> Node has 4 children
+ Tree building for CONNEL, Robert 89
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CONNEL, Leizie 95
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CONNEL, James 99
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CONNEL, William 103
> Node identified as class 'man'
> Node has 0 children
+ Tree building for AITKEN, Robert 109
> Node identified as class 'man'
> Node has 1 children
+ Tree building for AITKEN, James (3)
> Node identified as class 'man'
> Node has 5 children
+ Tree building for AITKEN, Janet 129
> Node identified as class 'man'
> Node has 0 children
+ Tree building for AITKEN, John 133
> Node identified as class 'man'
> Node has 0 children
+ Tree building 

...

> Node has 2 children
+ Tree building for BALLOCH, John (3)
> Node identified as class 'man'
> Node has 0 children
+ Tree building for BALLOCH, Martha 40
> Node identified as class 'man'
> Node has 1 children
+ Tree building for GILMORE, Jean 55
> Node identified as class 'man'
> Node has 0 children
+ Writing to visualisation dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS
+ Writing to trees dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS/^trees
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0234, --- BLAIR of Ladymuir.jpg.txt.PEO_REL.tsv
> Rows = 0, Columns = 10

+ Filtering PEO/REL data for SON/DAUG/CHILD
+ Removing duplicationes [of src_ref/src_linenum]
> Rows = 0, Columns = 10

+ No PEOPLA data, using default values in 'in_LWH'
+ No PEOPLA data, using default values in 'in_WIN'
> There are 0 root nodes
--------------------------------------------------------
+ Rea

> There are 1 root nodes
+ Tree building for SEMPILL, James (1)
> Node identified as class 'man'
> Node has 7 children
+ Tree building for SEMPILL, James (2)
> Node identified as class 'man'
> Node has 3 children
+ Tree building for SEMPILL, Margaret 109
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, James 118
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, Jean 123
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, Jean 37
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, Margaret 47
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, William 54
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, Elizabeth 70
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPILL, Alexander 78
> Node identified as class 'man'
> Node has 0 children
+ Tree building for SEMPIL

> Node identified as class 'man'
> Node has 0 children
+ Tree building for WILSON, Janet 163
> Node identified as class 'man'
> Node has 1 children
+ Tree building for BLAIR, Eliza 203
> Node identified as class 'man'
> Node has 0 children
+ Tree building for WILSON, Elizabeth 216
> Node identified as class 'man'
> Node has 0 children
+ Tree building for WILSON, William 228
> Node identified as class 'man'
> Node has 0 children
+ Tree building for WILSON, Margaret 239
> Node identified as class 'man'
> Node has 0 children
+ Tree building for WILSON, Robert 255
> Node identified as class 'man'
> Node has 0 children
+ Tree building for WILSON, James 301
> Node identified as class 'in_WIN'
> Node has 0 children
+ Tree building for SEMPILL, William 79
> Node identified as class 'man'
> Node has 0 children
+ Writing to visualisation dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS
+ Writing to trees dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS/^trees
----------

> Node identified as class 'man'
> Node has 0 children
+ Tree building for CRAWFURD, Agnes 658
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CRAWFURD, Mary 665
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CRAWFURD, Janet 672
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CRAWFURD, Agnes 698
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CRAWFURD, Andrew 82
> Node identified as class 'man'
> Node has 0 children
+ Tree building for CRAWFURD, Barbara 428
> Node identified as class 'man'
> Node has 5 children
+ Tree building for LOGAN, John 443
> Node identified as class 'man'
> Node has 0 children
+ Tree building for LOGAN, Andrew 451
> Node identified as class 'man'
> Node has 0 children
+ Tree building for LOGAN, George 458
> Node identified as class 'man'
> Node has 0 children
+ Tree building for LOGAN, William Crawfurd 463
> Node identified as class 'man'
> Node has 0 chi

> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Robert 391
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Duncan 396
> Node identified as class 'man'
> Node has 0 children
+ Tree building for ORR, Rankine 402
> Node identified as class 'in_WIN'
> Node has 0 children
+ Tree building for ORR, John 411
> Node identified as class 'in_WIN'
> Node has 0 children
+ Tree building for LANG, Elizabeth 436
> Node identified as class 'man'
> Node has 0 children
+ Writing to visualisation dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS
+ Writing to trees dir: /Users/lisahopcroft_tmp/Projects/History/tmpdat//04_^VIS/^trees
--------------------------------------------------------
+ Reading PEO/REL file
> /Users/lisahopcroft_tmp/Projects/History/tmpdat/03_^DAT/^0088, --- [===] DUNLOP of Lonehead [+].jpg.txt.PEO_REL.tsv
> Rows = 57, Columns = 10

+ Filtering PEO/REL data for SON/DAUG/CHILD
+ Removing duplicationes [of src_re